In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import math
import scipy
import scipy.io as si
from scipy import stats, signal
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm
import csv
import glob, os
import math

# Apply Multiple Transformation
Not a feature but used by other features

In [ ]:
class ApplyManyTransform(object):
    def apply(self, datas, meta):
        if datas.ndim >= 3:
            out = []
            for d in datas:
                out.append(self.apply_one(d, meta))

            return to_np_array(out)
        else:
            return self.apply_one(datas, meta)

# Creating  Windows
 Not a feature but used by other features

In [ ]:
class Slice:
    """
    Take a slice of the data on the last axis.
    e.g. Slice(1, 48) works like a normal python slice, that is 1-47 will be taken
    """
    def __init__(self, start, end=None):
        self.start = start
        self.end = end

    def get_name(self):
        return "slice%d%s" % (self.start, '-%d' % self.end if self.end is not None else '')

    def apply(self, data, meta=None):
        s = [slice(None),] * data.ndim
        s[-1] = slice(self.start, self.end)
        return data[s[0]]

class Windower:
    """
    Breaks the time-series data into N second segments, for example 60s windows
    will create 10 windows given a 600s segment. The output is the reshaped data
    e.g. (600, 120000) -> (600, 10, 12000)
    """
    def __init__(self, window_secs):
        self.window_secs = window_secs
        self.name = 'w-%ds' % window_secs if window_secs is not None else 'w-whole'

    def get_name(self):
        return self.name

    def apply(self, X, meta=None):
        if self.window_secs is None:
            return X.reshape([1] + list(X.shape))

        num_windows = 600 / self.window_secs
        samples_per_window = self.window_secs * 399
        samples_used = num_windows * samples_per_window
        samples_dropped = X.shape[-1] - samples_used
        X = Slice(samples_dropped).apply(X)
        out = np.split(X, num_windows, axis=X.ndim-1)
        out = np.asarray(out)
        return out

#Windower(100).apply(channels)

# Correlation

In [ ]:
def upper_right_triangle(matrix):
    accum = []
    for i in range(matrix.shape[0]):
        for j in range(i+1, matrix.shape[1]):
            accum.append(matrix[i, j])

    return np.asarray(accum)

class Eigenvalues(ApplyManyTransform):
    """
    Take eigenvalues of a matrix, and sort them by magnitude in order to
    make them useful as features (as they have no inherent order).
    """
    def get_name(self):
        return 'eigen'

    def apply_one(self, data, meta=None):
        w, v = np.linalg.eig(data)
        w = np.absolute(w)
        w.sort()
        return w
    
class CorrelationMatrix(ApplyManyTransform):
    """
    Calculate correlation coefficients matrix across all EEG channels.
    """
    def get_name(self):
        return 'corr-mat'

    def apply_one(self, data, meta=None):
        return np.corrcoef(data)
    
class UnitScaleFeat:
    """
    Scale across the first axis, i.e. scale each feature.
    """
    def get_name(self):
        return 'unit-scale-feat'

    def apply(self, data, meta=None):
        return preprocessing.scale(data.astype(np.float64), axis=0)


class UnitScale:
    """
    Scale across the last axis.
    """
    def get_name(self):
        return 'unit-scale'

    def apply(self, data, meta=None):
        return preprocessing.scale(data, axis=data.ndim-1)


class Correlation(ApplyManyTransform):
    """
    Correlation in the time domain. Calculate correlation co-efficients
    followed by calculating eigenvalues on the correlation co-efficients matrix.

    The output features are (upper_right_diagonal(correlation_coefficients), eigenvalues)

    Features can be selected/omitted using the constructor arguments.
    """
    def __init__(self, scale_option, with_corr=True, with_eigen=True):
        self.scale_option = scale_option
        self.with_corr = with_corr
        self.with_eigen = with_eigen
        assert scale_option in ('us', 'usf', 'none')
        assert with_corr or with_eigen

    def get_name(self):
        selections = []
        if self.scale_option != 'none':
            selections.append(self.scale_option)
        if not self.with_corr:
            selections.append('nocorr')
        if not self.with_eigen:
            selections.append('noeig')
        if len(selections) > 0:
            selection_str = '-' + '-'.join(selections)
        else:
            selection_str = ''
        return 'corr%s' % (selection_str)

    def apply_one(self, data, meta=None):
        data1 = data
        if self.scale_option == 'usf':
            data1 = UnitScaleFeat().apply(data1)
        elif self.scale_option == 'us':
            data1 = UnitScale().apply(data1)

        data1 = CorrelationMatrix().apply_one(data1)

        # patch nans
        data1[np.where(np.isnan(data1))] = -2

        if self.with_eigen:
            w = Eigenvalues().apply_one(data1)

        out = []
        if self.with_corr:
            data1 = upper_right_triangle(data1)
            out.append(data1)
        if self.with_eigen:
            out.append(w)

        for d in out:
            assert d.ndim == 1

        return np.concatenate(out, axis=0)
    
#Correlation('usf').apply_one(channels)

# Frequency Correlation

In [ ]:
class FreqCorrelation(ApplyManyTransform):
    """
    Correlation in the frequency domain. First take FFT with (start, end) slice options,
    then calculate correlation co-efficients on the FFT output, followed by calculating
    eigenvalues on the correlation co-efficients matrix.

    The output features are (fft, upper_right_diagonal(correlation_coefficients), eigenvalues)

    Features can be selected/omitted using the constructor arguments.
    """
    def __init__(self, start_hz, end_hz, option, use_phase=False, with_fft=False, with_corr=True, with_eigen=True):
        self.start_hz = start_hz
        self.end_hz = end_hz
        self.option = option
        self.with_fft = with_fft
        self.with_corr = with_corr
        self.with_eigen = with_eigen
        self.use_phase = use_phase
        assert option in ('us', 'usf', 'none', 'fft_in')
        assert with_corr or with_eigen

    def get_name(self):
        selections = []
        if self.option in ('us', 'usf', 'fft_in'):
            selections.append(self.option)
        if self.with_fft:
            selections.append('fft')
        if not self.with_corr:
            selections.append('nocorr')
        if not self.with_eigen:
            selections.append('noeig')
        if len(selections) > 0:
            selection_str = '-' + '-'.join(selections)
        else:
            selection_str = ''
        return 'freq-corr%s-%s-%s%s' % ('-phase' if self.use_phase else '', self.start_hz, self.end_hz, selection_str)

    def apply_one(self, data, meta=None):
        num_time_samples = data.shape[-1] if self.option != 'fft_in' else (data.shape[-1] - 1) * 2 # revert FFT shape change
        if self.start_hz == 1 and self.end_hz is None:
            freq_slice = Slice(self.start_hz, self.end_hz)
        else:
            # FFT range is from 0Hz to 101Hz
            def calc_index(f):
                return int((f / (meta.sampling_frequency/2.0)) * num_time_samples) if f is not None else num_time_samples
            freq_slice = Slice(calc_index(self.start_hz), calc_index(self.end_hz))
            # print data.shape, freq_slice.start, freq_slice.end
            # import sys
            # sys.exit(0)

        data1 = data
        if self.option != 'fft_in':
            data1 = FFT().apply(data1)
        data1 = freq_slice.apply(data1)
        if self.use_phase:
            data1 = np.angle(data1)
        else:
            data1 = Magnitude().apply(data1)
            data1 = Log10().apply(data1)

        data2 = data1
        if self.option == 'usf':
            data2 = UnitScaleFeat().apply(data2)
        elif self.option == 'us':
            data2 = UnitScale().apply(data2)

        data2 = CorrelationMatrix().apply_one(data2)

        if self.with_eigen:
            w = Eigenvalues().apply_one(data2)

        out = []
        if self.with_corr:
            data2 = upper_right_triangle(data2)
            out.append(data2)
        if self.with_eigen:
            out.append(w)
        if self.with_fft:
            data1 = data1.ravel()
            out.append(data1)
        for d in out:
            assert d.ndim == 1

        return np.concatenate(out, axis=0)


#FreqCorrelation(1, None, 'none').apply_one(channels)

# FFT
Not a feature but used by other features

In [ ]:
class FFT:
    """
    Apply Fast Fourier Transform to the last axis.
    """
    def get_name(self):
        return "fft"

    def apply(self, data, meta=None):
        axis = data.ndim - 1
        return np.fft.rfft(data, axis=axis)

# Magnitude abs
Not a feature but used by other features

In [ ]:
class Magnitude:
    """
    Take magnitudes of Complex data
    """
    def get_name(self):
        return "mag"

    def apply(self, data, meta=None):
        return np.abs(data)

# Frequency Binning
Not a feature but used by other features

In [ ]:
class FreqBinning(ApplyManyTransform):
    """
    Given spectral magnitude data, select a range of bins, and then choose a consolidation function
    to use to calculate each bin. The sum can be used, or the mean, or the standard deviation.

    NOTE(mike): Input for this transform must be from (FFT(), Magnitude())
    """
    def __init__(self, freq_ranges, func=None):
        self.freq_ranges = freq_ranges
        assert func is None or func in ('sum', 'mean', 'std')
        self.func = func

    def get_name(self):
        return 'fbin%s%s' % ('' if self.func is None else '-' + self.func, '-' + '-'.join([str(f) for f in self.freq_ranges]))

    def apply_one(self, X):
        num_channels = X.shape[0]
        num_time_samples = (X.shape[-1] - 1) * 2 # revert FFT shape change

        if self.func == 'mean':
            func = np.mean
        elif self.func == 'std':
            func = np.std
        else:
            func = np.sum

        # group into bins
        def binned_freq(data, out):
            prev = freq_ranges[0]
            for i, cur in enumerate(freq_ranges[1:]):
                prev_index = int(np.floor((prev / 399.6097561) * num_time_samples))
                cur_index = int(np.floor((cur / 399.6097561399) * num_time_samples))
                
                out[i] = func(data[prev_index:cur_index])
                prev = cur

        freq_ranges = self.freq_ranges
        out = np.empty((num_channels, len(freq_ranges) - 1,))
        for ch in range(num_channels):
            binned_freq(X[ch], out[ch])

        result = out[0]
        for i in range (1,len(out)):
            result = np.concatenate([result, out[i]]) 
        
        return result

#FreqBinning([0.5, 2.25, 4, 5.5, 7, 9.5, 12, 21, 30, 39, 48],'mean').apply_one(Magnitude().apply(FFT().apply(channels)))

# Apply Log10
Not a feature but used by other features

In [ ]:
class Log10:

    def get_name(self):
        return "log10"

    def apply(self, data, meta=None):
        indices = np.where(data <= 0)
        data[indices] = np.max(data)
        data[indices] = (np.min(data) * 0.1)
        return np.log10(data)

#Log10().apply(channels)

# FlattenChannels
Not a feature but used by other features    
Reshapes the data from (..., N_CHANNELS, N_FEATURES) to (..., N_CHANNELS * N_FEATURES)

In [ ]:
class FlattenChannels(object):

    def get_name(self):
        return 'fch'

    def apply(self, data, meta=None):
        if data.ndim == 2:
            return data.ravel()
        elif data.ndim == 3:
            s = data.shape
            return data.reshape((s[0], np.product(s[1:])))
        else:
            raise NotImplementedError()


# PIBSpectralEntropy

In [ ]:
class PIBSpectralEntropy(ApplyManyTransform):
    """
    Similar to the calculations in SpectralEntropy transform, but instead power-in-band
    is calculated over the given freq_ranges, finally Shannon entropy is calculated on that.
    The output is a single entropy value per-channel.

    NOTE(mike): Input for this transform must be from (FFT(), Magnitude())
    """
    def __init__(self, freq_ranges):
        self.freq_ranges = freq_ranges

    def get_name(self):
        return 'pib-spec-ent-%s' % '-'.join([str(f) for f in self.freq_ranges])

    def apply_one(self, data, meta=None):
        num_channels = data.shape[0]
        num_time_samples = float((data.shape[-1] - 1) * 2) # revert FFT shape change

        
        def norm(X):
            for i in range (len(X)):
                for j in range (len(X[i])):
                    X[i][j] /= X[i][j]+1e-12
            return X


        psd = data ** 2
        psd = norm(psd)

        # group into bins
        def binned_psd(psd, out):
            prev = freq_ranges[0]
            for i, cur in enumerate(freq_ranges[1:]):
                prev_index = int(np.floor((prev / 399.6097561) * num_time_samples))
                cur_index = int(np.floor((cur / 399.6097561399) * num_time_samples))
                out[i] = np.sum(psd[prev_index:cur_index])
                prev = cur

        freq_ranges = self.freq_ranges
        out = np.empty((num_channels, len(freq_ranges) - 1,))
        for ch in range(num_channels):
            binned_psd(psd[ch], out[ch])

        psd_per_bin = norm(out)

        def entropy_per_channel(psd):
            entropy_components = psd * np.log2(psd + 1e-12)
            entropy = -np.sum(entropy_components) / np.log2(psd.shape[-1])
            return entropy

        out = np.empty((num_channels,))
        for i, ch in enumerate(psd_per_bin):
            out[i] = entropy_per_channel(ch)

        return out
    

#PIBSpectralEntropy([0.25, 1, 1.75, 2.5, 3.25, 4, 5, 8.5, 12, 15.5, 19.5, 24]).apply_one(channels)
#PIBSpectralEntropy([0.25, 2, 3.5, 6, 15, 24]).apply_one(channels)
#PIBSpectralEntropy([0.25, 2, 3.5, 6, 15]).apply_one(channels)
#PIBSpectralEntropy([0.25, 2, 3.5]).apply_one(channels)
#PIBSpectralEntropy([6, 15, 24]).apply_one(channels)
#PIBSpectralEntropy([2, 3.5, 6]).apply_one(channels)
#PIBSpectralEntropy([3.5, 6, 15]).apply_one(channels)

# Shannon Entropy

In [ ]:
class ShannonEntropy(ApplyManyTransform):
    """
    Calculates Shannon entropy between the given frequency ranges.
    e.g. The probability density function of FFT magnitude is calculated, then
    given range [1,2,3], Shannon entropy is calculated between 1hz and 2hz, 2hz and 3hz
    in this case giving 2 values per channel.

    NOTE(mike): Input for this transform must be from (FFT(), Magnitude())
    """
    def __init__(self, freq_ranges, flatten=True):
        self.freq_ranges = freq_ranges
        self.flatten = flatten

    def get_name(self):
        return 'spec-ent-%s%s' % ('-'.join([str(f) for f in self.freq_ranges]), '-nf' if not self.flatten else '')

    def apply_one(self, fft_mag, meta=None):
        num_time_samples = (fft_mag.shape[-1] - 1) * 2 # revert FFT shape change

        X = fft_mag ** 2
        for ch in X:
            ch /= np.sum(ch + 1e-12)

        psd = X # pdf

        out = []

        #[0,1,2] -> [[0,1], [1,2]]
        for start_freq, end_freq in zip(self.freq_ranges[:-1], self.freq_ranges[1:]):
            start_index = int(np.floor((start_freq / 399.6097561) * num_time_samples))
            end_index = int(np.floor((end_freq / 399.6097561) * num_time_samples))
            selected = psd[:, start_index:end_index]

            entropies = - np.sum(selected * np.log2(selected + 1e-12), axis=selected.ndim-1) / np.log2(end_index - start_index)
            if self.flatten:
                out.append(entropies.ravel())
            else:
                out.append(entropies)

        if self.flatten:
            return np.concatenate(out)
        else:
            return np.asarray(out)

#ShannonEntropy([1,2,3]).apply_one(Magnitude().apply(FFT().apply(channels)))

# higuchi-fractal-dimension

In [ ]:
class HFD:
    
    def apply(self,epochs, Kmax = 2):
        '''
        Ported from https://www.mathworks.com/matlabcentral/fileexchange/30119-complete-higuchi-fractal-dimension-algorithm/content/hfd.m
        by Salai Selvam V
        '''
        result = list()
        
        for epoch in epochs:
            N = len(epoch)
    
            Lmk = np.zeros((Kmax,Kmax))
    
            #TODO: I think we can use the Katz code to refactor resampling the series
            for k in range(1, Kmax+1):
        
                for m in range(1, k+1):
               
                    Lmki = 0
            
                    maxI = int((N-m)/k)
            
                    for i in range(1,maxI+1):
                        Lmki = Lmki + np.abs(epoch[m+i*k-1]-epoch[m+(i-1)*k-1])
             
                    normFactor = (N-1)/(maxI*k)
                    Lmk[m-1,k-1] = normFactor * Lmki
    
            Lk = np.zeros((Kmax, 1))
    
            #TODO: This is just a mean. Let's use np.mean instead?
            for k in range(1, Kmax+1):
                Lk[k-1,0] = np.nansum(Lmk[range(k),k-1])/k/k

            lnLk = np.log(Lk) 
            lnk = np.log(np.divide(1., range(1, Kmax+1)))
    
            fit = np.polyfit(lnk,lnLk,1)  # Fit a line to the curve
     
            result.append(fit[0][0])   # Grab the slope. It is the Higuchi FD
    
        return np.array(result)
    
#HFD().apply(channels,2)

# Petrosian fractal dimension

In [ ]:
class PFD(ApplyManyTransform):
    """
    Petrosian fractal dimension per-channel

    Implementation derived from reading:
    http://arxiv.org/pdf/0804.3361.pdf
    F.S. Bao, D.Y.Lie,Y.Zhang,"A new approach to automated epileptic diagnosis using EEG
    and probabilistic neural network",ICTAI'08, pp. 482-486, 2008.
    """
    def get_name(self):
        return 'pfd'

    def pfd_for_ch(self, ch):
        diff = np.diff(ch, n=1, axis=0)

        asign = np.sign(diff)
        sign_changes = ((np.roll(asign, 1) - asign) != 0).astype(int)
        N_delta = np.count_nonzero(sign_changes)

        n = len(ch)
        log10n = np.log10(n)
        return log10n / (log10n + np.log10(n / (n + 0.4 * N_delta)))

    def apply_one(self, X, meta=None):
        return np.asarray([self.pfd_for_ch(ch) for ch in X])

#PFD().apply_one(channels)

# Hurst

In [ ]:
class Hurst:
    def apply(self,channels):
        result = list()
        for channel in channels:
            x = np.array(channel)
            x = x-x.mean()
            z = np.cumsum(x)
            r = np.array((np.maximum.accumulate(z) - np.minimum.accumulate(z))[1:])
            s = pd.expanding_std(x)[1:]
            s[np.where(s == 0)] = 1e-12
            r += 1e-12
            y_axis = np.log(r / s)
            x_axis = np.log(np.arange(1, len(y_axis) + 1))
            x_axis = np.vstack([x_axis, np.ones(len(x_axis))]).T
            m, b = np.linalg.lstsq(x_axis, y_axis)[0]
            result.append(m)
            result.append(b)
        return result

#Hurst().apply(channels)

# HJORTH Fractal Dimention

In [ ]:
def hjorthFD(channels, Kmax=3):
    result = list()
    for X in channels:
        L = []
        x = []
        N = len(X)
        for k in range(1,Kmax):
            Lk = []
            
            for m in range(k):
                Lmk = 0
                for i in range(1,int(math.floor((N-m)/k))):
                    Lmk += np.abs(X[m+i*k] - X[m+i*k-k])
                    
                Lmk = Lmk*(N - 1)/math.floor((N - m) / k) / k
                Lk.append(Lmk)
                
            L.append(np.log(np.nanmean(Lk)))   # Using the mean value in this window to compare similarity to other windows
            x.append([np.log(float(1) / k), 1])

        (p, r1, r2, s)= np.linalg.lstsq(x, L)  # Numpy least squares solution
	
        result.append(p[0])
    return result

#hjorthFD(channels)

# katz Fractal Dimention

In [ ]:
def katzFD(channels):
    result = list()
    for data in channels:
        n = len(data)-1
        L = np.hypot(np.diff(data), 1).sum() # Sum of distances
        d = np.hypot(data - data[0], np.arange(len(data))).max() # furthest distance from first point
        a = np.log10(n) / (np.log10(d/L) + np.log10(n))
        result.append(a)
    return result

#katzFD(channels)

# Power Spectral Density
 1. Largest amplitude value in PSD.
 2. Frequency of the largest peak.
 3. Second largest amplitude value in PSD.
 4. Frequency of the second largest peak.

In [ ]:
def psd(channels):
    result = list()
    for channel in channels:
        f, Pxx_den = signal.periodogram(list(channel))
        f = f.tolist()
        Pxx_den = Pxx_den.tolist()
        max_one = max(Pxx_den)
        freq_max_one = f[Pxx_den.index(max(Pxx_den))]
        Pxx_den.pop(Pxx_den.index(max(Pxx_den)))
        max_two = max(Pxx_den)
        freq_max_two = f[Pxx_den.index(max(Pxx_den))]
        result.append(max_one)
        result.append(freq_max_one)
        result.append(max_two)
        result.append(freq_max_two)
    return result

#psd(channels)

# Skewness and Kurtosis

In [ ]:
def moments(channels):  
    result = list()
    for data in channels:
        skewness = stats.skew(data, axis=0, bias=True)
        kurtosis = stats.kurtosis(data, axis=0, fisher=True, bias=True, nan_policy='propagate')
        result.append(skewness)
        result.append(kurtosis)
    return result

#moments(channels)

# Creating Csv

In [ ]:
training = list()
test = list()
for file in tqdm(glob.glob("dataset/Dog_5/*.mat")):
    if 'test' in file:
        test.append(file)
    else:
        training.append(file)

# Features of the channels combined

In [ ]:
attribute_list = list()
file_name = list()

for files in tqdm(test): 
    file_read = si.loadmat(files)
    #print file_read
    keys = file_read.keys()
    for i in range (len(keys)):
        if 'test' in keys[i]:
            channels = file_read[keys[i]][0][0][0]
            file_name.append(keys[i])
            feature_list = [
                Correlation('usf').apply_one(channels),FreqCorrelation(1, None, 'none').apply_one(channels),
                FreqBinning([0.5, 2.25, 4, 5.5, 7, 9.5, 12, 21, 30, 39, 48],'mean').apply_one(Magnitude().apply(FFT().apply(channels))),
                PIBSpectralEntropy([0.25, 1, 1.75, 2.5, 3.25, 4, 5, 8.5, 12, 15.5, 19.5, 24]).apply_one(channels),
                PIBSpectralEntropy([0.25, 2, 3.5, 6, 15, 24]).apply_one(channels),PIBSpectralEntropy([0.25, 2, 3.5, 6, 15]).apply_one(channels),
                PIBSpectralEntropy([0.25, 2, 3.5]).apply_one(channels),PIBSpectralEntropy([6, 15, 24]).apply_one(channels),
                PIBSpectralEntropy([2, 3.5, 6]).apply_one(channels), PIBSpectralEntropy([3.5, 6, 15]).apply_one(channels),
                ShannonEntropy([1,2,3]).apply_one(Magnitude().apply(FFT().apply(channels))),
                HFD().apply(channels,2), PFD().apply_one(channels), Hurst().apply(channels), hjorthFD(channels),
                katzFD(channels), psd(channels), moments(channels)
               ]
            
            result = feature_list[0]
            for i in range (1,len(feature_list)):
                result = np.concatenate([result, feature_list[i]]) 
            attribute_list.append(result)
            break

In [ ]:
train_df = pd.DataFrame()
train_df['name'] = file_name
train_df['attributes'] = attribute_list

In [ ]:
with open('d1features.csv', 'wb') as f:
    writer = csv.writer(f)
    for i in range(len(train_df)):
        feature_writer=list()
        feature_writer.append(train_df['name'][i])
        for val in train_df['attributes'][i]:
            feature_writer.append(val)
        
        writer.writerow(feature_writer)